In [ ]:
import pandas as pd
import configparser
import os
import jellyfish as jf
from fuzzywuzzy import utils
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import py_stringmatching as sm

config_file = '../config.ini'
config = configparser.ConfigParser()
config.read(config_file)

def calculate_score(distance, x, y):
    return round((1 - distance / max(len(x), len(y))) * 100)

def extract(query, choices, processor=utils.full_process, scorer=jf.levenshtein_distance, limit=2, distance=True):
    tmp = choices.to_frame('name')
    tmp['name'] = tmp['name'].apply(lambda x: processor(x))
    
    tmp['distance'] = tmp['name'].apply(lambda x: scorer(processor(query), str(x)))
    if distance:
        tmp['score'] = tmp.apply(lambda x: calculate_score(x['distance'], query, x['name']), axis=1)
    else:
        tmp['score'] = tmp['distance'].apply(lambda x: round(x * 100))
    
    tmp.sort_values(by=['score'], ascending=False, inplace=True)
    
    results = tmp[0:limit]
    best_results = []
    for key, value in results.iterrows():
        # print(value['ATC level name'])
        best_results.append((value['name'], value['score'], key))

    return best_results

In [ ]:
df = pd.read_csv(os.path.join(config['DEFAULT']['output_dir'],'bnf_code_clean.csv'))

In [ ]:
df_test = pd.read_csv(os.path.join('../data/test_analysis_set2.csv'))
atc_df = pd.read_csv(os.path.join('../data/rxnorm_atc_code_info.csv'))
atc_df.columns = ['i', 'rxcui', 'rxaui', 'sab', 'tty', 'ATC code', 'ATC level name', 'suppress']
atc_df.drop('i', axis=1, inplace=True)

In [ ]:
atc_df[:5]

In [ ]:
df_test['normalized_name'] = df_test['bnf_chemical_substance'].apply(lambda x: utils.full_process(x))

In [ ]:
df_test.info()

In [ ]:
s = atc_df['ATC level name']
tmp = s.to_frame('name')

In [ ]:
tmp['distance'] = tmp['name'].apply(lambda x: jf.jaro_distance('esomeprazole', str(x)))

In [ ]:
tmp[:6]

In [ ]:
tmp['distance'] = tmp['name'].apply(lambda x: jf.jaro_distance('esomeprazole', str(x)))
tmp['score'] = tmp['distance'].apply(lambda x: round(x*100))
#atc_df['score'] = atc_df.apply(lambda x: calculate_score(x['distance'], 'nizatidine', x['ATC level name']), axis=1)

In [ ]:
tmp.sort_values(by=['score'], ascending=False, inplace=True)
tmp[0:5]

In [ ]:
atc_df['distance'] = atc_df['ATC level name'].apply(lambda x: jf.jaro_distance('esomeprazole', x))
atc_df['score'] = atc_df['distance'].apply(lambda x: round(x*100))
#atc_df['score'] = atc_df.apply(lambda x: calculate_score(x['distance'], 'nizatidine', x['ATC level name']), axis=1)

In [ ]:
results = tmp[0:5]
results

In [ ]:
best_results = [];

for key,value in results.iterrows():
    #print(value['ATC level name'])
    best_results.append((value['name'], value['score'], key))
    
best_results

In [ ]:
extract('Nizatidine', s, scorer=jf.jaro_winkler, limit=5, distance=False)

In [ ]:
process.extract('nizatidine', s, scorer=fuzz.ratio, limit=5)

In [ ]:
scorer = sm.Cosine()
ws = sm.WhitespaceTokenizer()
qgram = sm.QgramTokenizer(prefix_pad='^', suffix_pad='!')
scorer.get_sim_score(ws.tokenize('nizatidine hydrochloride'),ws.tokenize('nizatidine'))

In [ ]:
scorer = sm.JaroWinkler()
hybrid = sm.MongeElkan(sim_func=scorer.get_sim_score)
ws = sm.WhitespaceTokenizer()
qgram = sm.QgramTokenizer(qval=2, prefix_pad='^', suffix_pad='!')
hybrid.get_raw_score(ws.tokenize('metoprolol tartarte'),ws.tokenize('metoprolol succinate'))

In [ ]:
scorer = PartialRatio()
ws = sm.WhitespaceTokenizer()
qgram = sm.QgramTokenizer(prefix_pad='^', suffix_pad='!')
scorer.get_sim_score('nizatidine hydrochloride','nizatidine')